# One-off Binary Tree Implementation


---

## Abstract

This document implements the one-off binary tree

---

## Example Data

In [ ]:
import numpy
from items import Item, ExampleItems 
# Item(sum={self.sum}, sum_of_squares={self.sum_of_squares}, num={self.num})"
e = ExampleItems()
items = e.calendar_items


## Merge Candidates

In the one-off binary tree, for any item, candidate merges are generated to be processed further:

$$
\begin{array}{c|ccccc}
\text{item} & v_0 & v_1 & v_2 & v_3 & v_4 \\
\hline
\text{candidate\_lo} &   & m(v_0, v_1) & m(v_1, v_2) & m(v_2, v_3) & m(v_3, v_4)  \\
\text{candidate\_hi} & m(v_0, v_1) & m(v_1, v_2) & m(v_2, v_3) & m(v_3, v_4) & m(v_4, v_5)
\end{array}
$$

In [ ]:
tmp = []
for n in range(1, len(items)):
    tmp.append(Item(items[n - 1], items[n]))

cand_lo = [None] + tmp
cand_hi = tmp + [None]
cand = [cand_lo, cand_hi]

## Merge Candidate Preference

Item $v_1$ has two candidate merges and chooses the better one (marked bold in the table), the same with $v_2$.  
If their choices agree, we get a "happy couple":

$$
\begin{array}{c|cc}
\text{item}  & v_1 & v_2 \\
\hline
\text{candidate\_lo} & m(v_0, v_1) & \bf{m(v_1, v_2)} \\
\text{candidate\_hi} & \bf{m(v_1, v_2)} & m(v_2, v_3) 
\end{array}
$$

In [ ]:
lo_is_better = []
for n in range(len(items)):
    lo_item = cand_lo[n]
    hi_item = cand_hi[n]

    
    if lo_item is None:
        lo_is_better.append(False)
    elif hi_item is None:
        lo_is_better.append(True)
    else:
        lo_is_better.append(lo_item.quality > hi_item.quality)

import numpy as np
lo_is_better = np.array([lo_is_better])


A Couple is written as $C_j = [v_j, v_{j+1}]$.

In the other case, we get a "midsummer night's dream":  
$v_1$ loves $v_2$, but $v_2$ loves $v_3$, etc.

$$
\begin{array}{c|cc}
\text{item}  & v_1 & v_2 \\
\hline
\text{candidate\_lo} & m(v_0, v_1) & m(v_1, v_2) \\
\text{candidate\_hi} & \bf{m(v_1, v_2)} & \bf{m(v_2, v_3)} 
\end{array}
$$

In [ ]:


# Create other boolean arrays
hi_is_better = ~lo_is_better  # hi_is_better is the negation of lo_is_better
is_couple_begin = np.zeros(len(items), dtype=bool)
belongs_couple = np.zeros(len(items), dtype=bool)
is_triple_begin = np.zeros(len(items), dtype=bool)
belongs_triple = np.zeros(len(items), dtype=bool)
belongs_up_chain = np.zeros(len(items), dtype=bool)
belongs_down_chain = np.zeros(len(items), dtype=bool)


# Initialize previous values
previous_index = 0
previous_lo_is_better = lo_is_better[0]
previous_hi_is_better = hi_is_better[0]

# Loop from index 1 to the last element
for current_index in range(1, size):
    current_lo_is_better = lo_is_better[current_index]
    current_hi_is_better = hi_is_better[current_index]

    if previous_hi_is_better:
        if current_lo_is_better:
            # We have found a happy couple
            is_couple_begin[previous_index] = True
            belongs_couple[previous_index] = True
            belongs_couple[current_index] = True
        else:
            # We have found a love chain leading upwards (A loves B, but B loves C, but C loves D ...)
            belongs_up_chain[previous_index] = True
    else:
        # previous_lo_is_better: the previous item does not like the current one
        if current_lo_is_better:
            # A love chain in the opposite direction
            belongs_down_chain[current_index] = True
    
    # Update previous values
    previous_index = current_index
    previous_lo_is_better = current_lo_is_better
    previous_hi_is_better = current_hi_is_better

result_vector = belongs_up_chain | belongs_down_chain | belongs_couple  # Element-wise OR
assert np.all(result_vector)  # True if all elements of result_vector are True


### Chain Resolution

Above, items $v_{1 \dots 3}$ form a chain. Chains can be of any length.  
A chain is terminated by a happy Couple.  
However, we can reduce this to three cases.

---

#### Two Couples

A chain of length 2 and termination at both ends, with $m(v_i, v_j)$ written as $(i, j)$:

$$
\begin{array}{c|ccccccc}
\text{item}  & 1 & 2 & 3 & 4 & 5 & 6 & 7 \\
\hline
\text{candidate\_lo} & (0, 1) & \bf{(1, 2)} & (2, 3) & (3, 4) & (4, 5) & \bf{(5, 6)} \\
\text{candidate\_hi} & \bf{(1, 2)} & (2, 3) & \bf{(3, 4)} & \bf{(4, 5)} & \bf{(5, 6)} & (6, 7)
\end{array}
$$

The items of a chain of length 2 settle into a couple.

---

In [ ]:
# merge chain items into couple whenever possible
for current_index in range(1, len(items):
    previous_index = current_index -1
    if belongs_down_chain[previous_index] and belongs_down_chain[current_index] = True:
        # found two chain elements
        belongs_down_chain[previous_index] = false
        belongs_down_chain[current_index] = false
        # force them into a couple
        is_couple_begin[previous_index] = True
        belongs_couple[previous_index] = True
        belongs_couple[current_index] = True

    if belongs_up_chain[previous_index] and belongs_up_chain[current_index] = True:
        # found two chain elements
        belongs_up_chain[previous_index] = false
        belongs_up_chain[current_index] = false
        # force them into a couple
        is_couple_begin[previous_index] = True
        belongs_couple[previous_index] = True
        belongs_couple[current_index] = True



#### Triples

A chain of one element is called a Triple, written as:

$$
T_j = [v_j, v_{j+1}, v_{j+2}].
$$

After settling, there are only Couples and Triples. Between Triples, there can exist any number of Couples or none.

Any nonempty sequence of Triples $Q$ starts with an Even Triple and ends with an Odd Triple.

A Triple Pair can also be written 

in terms of $\bar{j}$ indices $i, {i'} \in [0, 1, \dots, \bar{e}]$:

$$
P = (T_{a(k)}, T_{a(k+1)}) = P_{i{i'}}, \quad a(k) = 2i, \quad a(k+1) = 2{i'} - 1
$$



In [ ]:
# confirm there are only chains with on element left 
for current_index in range(1, len(items)):
    previous_index = current_index -1
    assert not (belongs_up_chain[previous_index] and belongs_up_chain[current_index])
    assert not (belongs_down_chain[previous_index] and belongs_down_chain[current_index])

# confirm each single element is attached to a couple
for current_index in range(1, len(items)):
    if belongs_up_chain[current_index]:
        assert current_index < len(items)-2
        assert belongs_couple[current_index + 1] and belongs_couple[current_index + 2]
       
    if belongs_down_chain[current_index]:
        assert current_index > 1
        assert belongs_couple[current_index - 1] and belongs_couple[current_index - 2]

assert not belongs_up_chain[-1]
assert not belongs_down_chain[0]

In [ ]:
for current_index in range(0, len(items)-1):
    if belongs_up_chain[current_index]:
        is_triple_begin[current_index]=true
        belongs_triple[current_index:current_index+2]=true
    if belongs_down_chain[current_index]:
        is_triple_begin[current_index-2]=true
        belongs_triple[current_index-2:current_index]=true      

In [ ]:
# the triple start indices are Even - Odd - Even ...
triple_begin_must_be_even = true
triple_begin_must be_odd = false
for current_index in range(0, len(items)):
    if is_triple_begin[current_index]:
        current_index_is_even = current_index%2==0
        current_index_is_odd = current_index%2==1
        assert not (current_index_is_even and triple_begin_must_be_odd)
        assert not (current_index_is_odd and triple_begin_must_be_even)
        triple_begin_must_be_even = not triple_begin_must_be_even
        triple_begin_must_be_odd = not triple_begin_must_be_odd

---

### Sequence of Triple Pairs

The sequence of Triple Pairs can be written as:

$$
R: P_{i(k){i'}(k)}, \quad i(k) < {i'}(k) < i(k+1), \quad i, {i'} \in [0, 1, \dots, \bar{e}]
$$

with

$$
P_{i{i'}} = (T_{2i}, T_{2{i'} - 1})
$$


---

### Defects

A change in $t_{j-1} \ne t_j$ is called a Defect, denoted $D_j$.

A series of consecutive Couples translates into a section of constant $t$ values.  
A Triple of items $T_j$ corresponds one-to-one to a Defect in $t$.  
There are four types of Defects:

$$
(t_{j-1}, t_j) \text{ with values } \quad (-1, 0), \quad (0, 1), \quad (0, -1), \quad (1, 0).
$$

---

In [ ]:
c=1


### Triples Relative to the Binary Grid

How to merge a Triple also depends on its position relative to the binary grid. We set apart:  
- The **Even Triple** $T_j$ with an even index, and  
- The **Odd Triple** otherwise.

The sequence of Triples within a tree row $v$ is given by:

$$
Q: \{ T_{a(k)}, k = 0 \dots \text{number of Triples} - 1, \quad a(j) + 2 < a(j+1), 0 \le j \le e \}
$$

In [ ]:
d=1

### Triples to Merge

Denoting a Single as an item being transferred to the next-higher row alone:

$$
S_j = (v_j),
$$

assign the full merge to
$$
 T'_j: (v_j, (v_{j+1}, v_{j+2})) = ((v_j, v_{j+1}), v_{j+2})
$$

Compare the quality of the $S_j C_{j+1}$ merge and the quality of the $C_j S_{j+2}$ merge
and assign the merge with the better quality to

$$
K_j := 
\begin{cases} 
S_j C_{j+1} & \text{if } q(S_j C_{j+1}) > q(C_j S_{j+2}) \\
C_j S_{j+2} & \text{otherwise}
\end{cases}
$$

In [ ]:
e=1

### Merging a Triple Pair

There are two ways to merge a Triple Pair $P: (T_{2i}, T_{2{i'} - 1})$, with $i < i'$:  
1. As $(T'_{2i}, K_{2{i'}-1})$,  
2. As $(K_{2i}, T'_{2{i'}-1})$.  

Compare the two merge pairs $(T'_{2i}, K_{2{i'}-1})$ and $(K_{2i}, T'_{2{i'}-1})$ 
and call the merge with the better quality the **Triple Merge Pair**:

$$
Z_{i{i'}} = (W_{2i}, W_{2{i'}-1}) := 
\begin{cases} 
(T'_{2i}, K_{2{i'}-1}) & \text{if } q(T'_{2i}, K_{2{i'}-1}) > q(K_{2i}, T'_{2{i'}-1}) \\ 
(K_{2i}, T'_{2{i'}-1}) & \text{otherwise}
\end{cases}
$$

---


In [ ]:
f=1

### Sequence of Merge Triple Pairs

We define the sequence of Merge Triple Pairs $V$, which is isomorphic to the sequence of Triple Pairs $P$:

$$
V: Z_{i(k){i'}(k)}, \quad i(k) < {i'}(k) < i(k+1), \quad i, {i'} \in [0, 1, \dots, \bar{e}]
$$

---

### Sequence of Merge Triples

We define the sequence of Merge Triples $T$, which is isomorphic to the sequence of Triples $Q$:

$$
T: \{ W_{a(k)}, \quad k = 0, \dots, \text{number of Triples} - 1 \}
$$

The sequence $T$ then translates into the $t$ vector.
The complete $t$ vector then allows to merge the Couples:


In [ ]:
g=1

### Couples to Merge

A Couple $C_j$ with an even index is merged just like in the Binary Tree:

$$
t^1_j = 0: \\
j^1_i = (j^0_{2i}, j^0_{2i+1}).
$$

For a Couple $C_j$ with an odd index, there is a choice: It is either merged according to:

$$
t^1_j = 1: \\
j^1_i = (j^0_{2i-1}, j^0_{2i}),
$$

or

$$
t^1_j = -1: \\
j^1_i = (j^0_{2i+1}, j^0_{2i+2}).
$$


---

In [ ]:
h=1

## One-off Binary Tree Definition
With $i = 2j$:

$$
r(\bar{v}, -) = r(v_{2i + t_i}, -), \quad r(\bar{v}, +) = r(v_{2(i+1)+1+t_{i+1}}, +)
$$

with unit shifts:

$$
t_0 = 0, \quad t_i \in \{-1, 0, 1\}
$$

obeying:

$$
\left| t_{i+1} - t_i \right| \le 1.
$$